In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [12]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [13]:
# Load and preprocess the data
df = pd.read_csv('panama_dataset.csv')
df.dropna(inplace=True)

s1 = MinMaxScaler(feature_range=(0, 1))
s2 = MinMaxScaler(feature_range=(0, 1))

In [14]:
# Selecting only the target variable and scaling
ys = s2.fit_transform(df[['target_variable']])

In [15]:
# Scaling the input features
Xs = s1.fit_transform(df.drop(columns=['target_variable']))
window = 1
num_features = Xs.shape[1]

In [16]:
# Reshape the input features to have shape (None, 5, num_features)
Xs_reshaped = np.array([Xs[i:i+window] for i in range(len(Xs)-window+1)])

# Slice the target variable to match the length of Xs_reshaped
ys_sliced = ys[window-1:]

In [17]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(Xs_reshaped, ys_sliced, test_size=0.2, shuffle=False)

In [18]:
def dual_attention_lstm(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
    attention1 = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention1 = tf.keras.layers.GlobalAveragePooling1D()(attention1)
    #attention2 = tf.keras.layers.Attention()([attention1, lstm_out])
    #attention2 = tf.keras.layers.GlobalAveragePooling1D()(attention2)
    
    outputs = tf.keras.layers.Dense(1, activation='linear')(attention1)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = dual_attention_lstm((X_train.shape[1], num_features))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

In [19]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32)

loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test MAE: {mae}')

y_pred = s2.inverse_transform(model.predict(X_test))
y_test = s2.inverse_transform(y_test)

Epoch 1/50
1202/1202 [==============================] - 8s 4ms/step - loss: 0.0086 - mae: 0.0718
Epoch 2/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0067 - mae: 0.0660
Epoch 3/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0065 - mae: 0.0652
Epoch 4/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0064 - mae: 0.0646
Epoch 5/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0064 - mae: 0.0645
Epoch 6/50
1202/1202 [==============================] - 4s 4ms/step - loss: 0.0064 - mae: 0.0643
Epoch 7/50
1202/1202 [==============================] - 4s 4ms/step - loss: 0.0063 - mae: 0.0641
Epoch 8/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0063 - mae: 0.0638
Epoch 9/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0062 - mae: 0.0636
Epoch 10/50
1202/1202 [==============================] - 4s 3ms/step - loss: 0.0062 - mae: 0.0635
Epoch 11/50
1202/1202 [======

In [20]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse / np.mean(y_pred)) * 100

print("RMSE: ", rmse)
print("MAE", mae)
print("MAPE", mape)
print("MSE", mse)
print("NRMSE", nrmse)

RMSE:  158.47970359880955
MAE 125.04525375972496
MAPE 10.514877867243715
MSE 25115.816452766525
NRMSE 12.967529399017586
